In [11]:
from tensorflow import keras
from PIL import Image
from torchvision import models, transforms
import torch.nn as nn
import torch.optim as optim
import torch
import random
from torchvision import models, transforms
from numpy.linalg import norm
import numpy as np
import math
import time

In [12]:
import sys
root = '../../'
sys.path.append(root)
from HelpfulFunctions.batchCreation import createBatch
from HelpfulFunctions.metrics import meanAveragePrecision
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
X_train = np.load( root + "Features/train_features_vgg16_cifar10.npy" ) # Shape = (45000, 4096)
X_train_tensor = torch.tensor(X_train)
y_train = np.load( root + "Features/train_labels_vgg16_cifar10.npy" ) # Shape = (45000,)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_train = torch.nn.functional.one_hot(y_train_tensor, num_classes=10) # Shape = (45000, 10)

X_test = np.load( root + "Features/test_features_vgg16_cifar10.npy" ) # Shape = (10000, 4096)
X_test_tensor = torch.tensor(X_test)
y_test = np.load( root + "Features/test_labels_vgg16_cifar10.npy" ) # Shape = (10000,)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)
y_test = torch.nn.functional.one_hot(y_test_tensor, num_classes=10) # Shape = (10000, 10)

In [16]:
y_train.shape

torch.Size([45000, 10])

In [ ]:
class CustomNN(nn.Module):
    def __init__(self):
        super(CustomNN, self).__init__()
        self.fc_layers = nn.Sequential(
            nn.Linear(4096, 1024),  # First fully connected layer
            nn.ReLU(),
            nn.Linear(1024, 32),    # Second fully connected layer to reduce to 4000
        )

        # Initialize weights and biases from gaussian distribution
        for layer in self.fc_layers:
            if isinstance(layer, nn.Linear):
                nn.init.normal_(layer.weight, mean=0.0, std=0.01)  # Initialize weights based on paper
                nn.init.normal_(layer.bias, mean=0.0, std=0.01)    # Initialize biases based on paper

    def forward(self, x):
        return self.fc_layers(x)

In [ ]:
model = CustomNN()

In [ ]:
class DPSHLoss(torch.nn.Module):
    def __init__(self, train_size, n_classes, bit):
        super(DPSHLoss, self).__init__()
        self.U = torch.zeros(train_size, bit).float().to(device)
        self.Y = torch.zeros(train_size, n_classes).float().to(device)

    def forward(self, u, y, ind, eta):
        self.U[ind, :] = u.data
        self.Y[ind, :] = y.float()

        s = (y @ self.Y.t() > 0).float()
        inner_product = u @ self.U.t() * 0.5

        likelihood_loss = (1 + (-(inner_product.abs())).exp()).log() + inner_product.clamp(min=0) - s * inner_product

        likelihood_loss = likelihood_loss.mean()

        quantization_loss = eta * (u - u.sign()).pow(2).mean()

        return likelihood_loss + quantization_loss

In [ ]:
def train_val(device, train_size, n_classes, bit, epoch):

    optimizer = optim.Adam(model.parameters(), lr=0.001)

    criterion = DPSHLoss(train_size = 45000, n_classes = 10, bit = 48)

    Best_mAP = 0

    for epoch in range(epoch):

        current_time = time.strftime('%H:%M:%S', time.localtime(time.time()))

        print("%s[%2d/%2d][%s] bit:%d, dataset:%s, training...." % (
            "DPSH", epoch + 1, epoch, current_time, bit, "CIFAR"), end="")

        

        train_loss = 0
        for image, label, ind in train_loader:
            image = image.to(device)
            label = label.to(device)

            optimizer.zero_grad()
            u = net(image)

            loss = criterion(u, label.float(), ind, config)
            train_loss += loss.item()

            loss.backward()
            optimizer.step()

        train_loss = train_loss / len(train_loader)

        print("\b\b\b\b\b\b\b loss:%.5f" % (train_loss))

        if (epoch + 1) % config["test_map"] == 0:
            Best_mAP = validate(config, Best_mAP, test_loader, dataset_loader, net, bit, epoch, num_dataset)
